# [570. Managers with at Least 5 Direct Reports](https://leetcode.com/problems/managers-with-at-least-5-direct-reports/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Employee

<pre>+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| name        | varchar |
| department  | varchar |
| managerId   | int     |
+-------------+---------+</pre>
id is the primary key (column with unique values) for this table.
Each row of this table indicates the name of an employee, their department, and the id of their manager.
If managerId is null, then the employee does not have a manager.
No employee will be the manager of themself.
 

Write a solution to find managers with at least five direct reports.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Employee table:
<pre>+-----+-------+------------+-----------+
| id  | name  | department | managerId |
+-----+-------+------------+-----------+
| 101 | John  | A          | None      |
| 102 | Dan   | A          | 101       |
| 103 | James | A          | 101       |
| 104 | Amy   | A          | 101       |
| 105 | Anne  | A          | 101       |
| 106 | Ron   | B          | 101       |
+-----+-------+------------+-----------+</pre>
Output: 
<pre>+------+
| name |
+------+
| John |
+------+</pre>

In [1]:
# Pandas schema

import pandas as pd

data = [[101, 'John', 'A', None], [102, 'Dan', 'A', 101], [103, 'James', 'A', 101], [104, 'Amy', 'A', 101], [105, 'Anne', 'A', 101], [106, 'Ron', 'B', 101]]
employee = pd.DataFrame(data, columns=['id', 'name', 'department', 'managerId']).astype({'id':'Int64', 'name':'object', 'department':'object', 'managerId':'Int64'})

In [2]:
# to pyspark schema

import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import *


schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("department", StringType(), True),
    StructField("managerId", IntegerType(), True)
])

In [3]:
spark = SparkSession.builder.getOrCreate()

employee_df = spark.createDataFrame(data,schema=schema)
employee_df.show(truncate=False)

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/11/05 12:06:36 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/11/05 12:06:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/05 12:06:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---+-----+----------+---------+
|id |name |department|managerId|
+---+-----+----------+---------+
|101|John |A         |null     |
|102|Dan  |A         |101      |
|103|James|A         |101      |
|104|Amy  |A         |101      |
|105|Anne |A         |101      |
|106|Ron  |B         |101      |
+---+-----+----------+---------+


In [4]:
# in Spark Dataframe

employee_df.select('name').filter(F.col('id').isin(
        list(employee_df.select('managerId').groupBy('managerId').agg(F.count(F.col('managerId')).alias('count')).filter(
        F.col('count') >= 5).select('managerId').rdd.map(lambda x:x[0]).collect())
)).show()

+----+
|name|
+----+
|John|
+----+


In [5]:
# in Spark SQL
employee_df.createOrReplaceTempView('employee')
spark.sql('''
select name
from employee
where id in (select managerId from employee group by managerId having count(managerId)>=5)
''').show()

+----+
|name|
+----+
|John|
+----+


In [6]:
spark.stop()